In [1]:
import pickle
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import torch.nn as nn
import torch.optim as optim
import zipfile

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# Load the training data
with open('train.pkl', 'rb') as f:
    train_data = pickle.load(f)

# Load the test data
with open('test_no_target.pkl', 'rb') as f:
    test_data = pickle.load(f)

In [ ]:
nuty_list = []
sum = 0
for elem in train_data:
    x = elem[0]
    for nuta in x:
        if nuta in nuty_list:
            continue
        else:
            nuty_list.append(nuta)
            sum+=1

In [ ]:
max_val = max(nuty_list)
min_val = min(nuty_list)
print(min_val, max_val)

In [ ]:
train_data_norm = []
test_data_norm = []

for i in range (len(train_data)):
    normalized_sequence = (train_data[i][0] - min_val) / (max_val - min_val)
    train_data_norm.append((normalized_sequence, train_data[i][1]))

for i in range (len(test_data)):
    normalized_sequence = (test_data[i][0] - min_val) / (max_val - min_val)
    test_data_norm.append((normalized_sequence, test_data[i][1]))

print(len(train_data_norm))
print(len(test_data_norm))

# print(train_data_norm[5])

In [4]:
# MusicDataset to handle sequences and labels
class MusicDataset(Dataset):
    def __init__(self, data, labels=None):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if self.labels is not None:
            return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)
        return torch.tensor(self.data[idx], dtype=torch.float32)

In [5]:
# Function to pad sequences and return batch data
def collate_fn(batch):
    sequences, labels = zip(*batch)
    lengths = torch.tensor([len(seq) for seq in sequences])
    padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=-2)
    return padded_sequences, torch.stack(labels), lengths

In [6]:
# Prepare the training and validation datasets
train_sequences, train_labels = zip(*train_data)
full_dataset = MusicDataset(train_sequences, train_labels)

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

# Prepare the test dataset and loader
test_dataset = MusicDataset(test_data)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

In [7]:
# Define the ComposerClassifier model
class ComposerClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, dropout):
        super(ComposerClassifier, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True, dropout=dropout, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # * 2 because bidirectional
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, lengths):
        packed_embedded = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        output, output_lengths = pad_packed_sequence(packed_output, batch_first=True)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))  # Concatenate hidden from both directions
        return self.fc(hidden)


In [8]:
# Hyperparameters
input_dim = 1  # Tylko nuty
hidden_dim = 324
output_dim = 5  # Liczba kompozytorów
n_layers = 2
dropout = 0.3

In [9]:
model = ComposerClassifier(input_dim, hidden_dim, output_dim, n_layers, dropout).to(device)
# Training setup
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [10]:
def calculate_accuracy(predictions, labels):
    _, preds = torch.max(predictions, dim=1)
    correct = (preds == labels).float()
    acc = correct.sum() / len(correct)
    return acc

In [11]:
# Training loop
n_epochs = 5

model.train()
for epoch in range(n_epochs):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for sequences, labels, lengths in train_loader:
        sequences, labels, lengths = sequences.to(device), labels.to(device), lengths.to(device)
        optimizer.zero_grad()
        sequences = sequences.unsqueeze(-1)


        predictions = model(sequences, lengths)
        loss = criterion(predictions, labels)
        acc = calculate_accuracy(predictions, labels)
        loss.backward()
        optimizer.step()


        epoch_loss += loss.item()
        epoch_acc += acc.item()
    epoch_loss /= len(train_loader)
    epoch_acc /= len(train_loader)

    val_loss = 0
    val_acc = 0
    model.eval()
    with torch.no_grad():
        for sequences, labels, lengths in val_loader:
            sequences, labels, lengths = sequences.to(device), labels.to(device), lengths.to(device)
            sequences = sequences.unsqueeze(-1)


            predictions = model(sequences, lengths)
            loss = criterion(predictions, labels)
            acc = calculate_accuracy(predictions, labels)


            val_loss += loss.item()
            val_acc += acc.item()
    val_loss /= len(val_loader)
    val_acc /= len(val_loader)

    print(f'Epoch {epoch+1}, Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

model.eval()


Epoch 1, Train Loss: 1.2149, Train Acc: 0.5562, Val Loss: 1.1042, Val Acc: 0.5630
Epoch 2, Train Loss: 1.0657, Train Acc: 0.6025, Val Loss: 1.0013, Val Acc: 0.6140


KeyboardInterrupt: 